In [ ]:
import os
import pandas as pd

# =========================
# הגדרות
# =========================
EXCEL_PATH = "With Photos Only_Naiv.xlsx"
TAGS_DIR = "tags"
FILENAME_COLUMN = "שם הקובץ"
OUTPUT_CSV = "With_Photos_Only_with_locations.csv"

VALID_EXTENSIONS = {".jpg", ".jpeg", ".tif", ".tiff"}


# =========================
# עזר: ניקוי שם קובץ (בלי סיומת, lowercase)
# =========================
def normalize_name(filename: str) -> str:
    name = os.path.splitext(str(filename))[0]
    return name.lower().strip()


# =========================
# שלב 1: סריקה רקורסיבית של tags
# =========================
print("🔍 Scanning tags directory...")

all_images = []

for root, _, files in os.walk(TAGS_DIR):
    for file in files:
        ext = os.path.splitext(file)[1].lower()
        if ext in VALID_EXTENSIONS:
            full_path = os.path.join(root, file)
            all_images.append({
                "filename": file.lower(),
                "path": os.path.abspath(full_path)
            })

print(f"✅ Found {len(all_images)} image files")


# =========================
# שלב 2: טעינת הדאטה
# =========================
df = pd.read_excel(EXCEL_PATH)

if FILENAME_COLUMN not in df.columns:
    raise ValueError(f"Column '{FILENAME_COLUMN}' not found in Excel")

df["_search_key"] = df[FILENAME_COLUMN].apply(normalize_name)


# =========================
# שלב 3: חיפוש התאמות
# =========================
print("🔎 Matching images to rows...")

all_matches = []

for key in df["_search_key"]:
    matches = [
        img["path"]
        for img in all_images
        if key and key in img["filename"]
    ]
    all_matches.append(matches)

# קביעת מספר העמודות הדינמי
max_matches = max(len(m) for m in all_matches)
print(f"📌 Max matches per row: {max_matches}")


# =========================
# שלב 4: יצירת עמודות location_X
# =========================
for i in range(max_matches):
    col_name = f"location_{i + 1}"
    df[col_name] = [
        matches[i] if i < len(matches) else ""
        for matches in all_matches
    ]


# =========================
# ניקוי עמודת עזר
# =========================
df.drop(columns=["_search_key"], inplace=True)


# =========================
# שלב 5: שמירה ל-CSV
# =========================
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

print(f"✅ Done! Output saved to: {OUTPUT_CSV}")
